# 연합뉴스 API
- index =  rens_testa
- 내부_ip : 192.168.1.66 , 외부_ip: 1.225.38.51
- port : 9206

In [16]:
from pandasticsearch import Select
from elasticsearch import Elasticsearch
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()

In [17]:
# !pip install elasticsearch==7.13.1

## ES

### Elasticsearch연결 및 기본코드

In [18]:
#엘라스틱서치 연결
es = Elasticsearch(hosts='192.168.1.66', port=9206, timeout=30, max_retries=10, retry_on_timeout=True)

In [19]:
body= {
  "query": {"match_all": {
  
}}}
result = es.search(index='rens_test', body=body, request_timeout=30)

In [20]:
Select.from_dict(result).to_pandas()

,_index,_type,_id,_score,YNewsML.Header.SendTime,YNewsML.Header.Action,YNewsML.Header.ServiceType,YNewsML.Header.ContentID,YNewsML.Header.SendDate,YNewsML.Metadata.Category.code,...,YNewsML.NewsContent.MultiMedia.DataFile.mimetype,YNewsML.NewsContent.MultiMedia.DataFile.type,YNewsML.NewsContent.MultiMedia.DataFile.content,YNewsML.NewsContent.MultiMedia.DataFile.desc,YNewsML.NewsContent.LangType,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,send_timestamp,YNewsML.Metadata.Category,YNewsML.Metadata.Class.ClassCode
0,rens_test,_doc,PYH20210331117600013,1.0,130204,C,PYH0,PYH20210331117600013,20210331,03,...,image/jpeg,P4,PYH2021033111760001300_P4.jpg,PREVIEW,KR,여의서로 벚꽃길 오늘붜 전면 통제,여의서로 벚꽃길 오늘붜 전면 통제\r\n (서울=연합뉴스) 윤동진 기자 = 3...,2021-03-31T13:02:04,NaN,NaN
1,rens_test,_doc,PYH20210331117700013,1.0,130207,C,PYH0,PYH20210331117700013,20210331,03,...,image/jpeg,P4,PYH2021033111770001300_P4.jpg,PREVIEW,KR,여의서로 벚꽃길 오늘붜 전면 통제,여의서로 벚꽃길 오늘붜 전면 통제\r\n (서울=연합뉴스) 윤동진 기자 = 3...,2021-03-31T13:02:07,NaN,NaN
2,rens_test,_doc,PYH20210331117800013,1.0,130210,C,PYH0,PYH20210331117800013,20210331,03,...,image/jpeg,P4,PYH2021033111780001300_P4.jpg,PREVIEW,KR,여의서로 벚꽃길 오늘붜 전면 통제,여의서로 벚꽃길 오늘붜 전면 통제\r\n (서울=연합뉴스) 윤동진 기자 = 3...,2021-03-31T13:02:10,NaN,NaN
3,rens_test,_doc,PYH20210331117900013,1.0,130231,C,PYH0,PYH20210331117900013,20210331,03,...,image/jpeg,P4,PYH2021033111790001300_P4.jpg,PREVIEW,KR,여의서로 벚꽃길 오늘붜 전면 통제,여의서로 벚꽃길 오늘붜 전면 통제\r\n (서울=연합뉴스) 윤동진 기자 = 3...,2021-03-31T13:02:31,NaN,NaN
4,rens_test,_doc,PYH20210331118000013,1.0,130234,C,PYH0,PYH20210331118000013,20210331,03,...,image/jpeg,P4,PYH2021033111800001300_P4.jpg,PREVIEW,KR,여의서로 벚꽃길 통제,여의서로 벚꽃길 통제\r\n (서울=연합뉴스) 윤동진 기자 = 31일 오전 통...,2021-03-31T13:02:34,NaN,NaN
5,rens_test,_doc,PYH20210331118100013,1.0,130242,C,PYH0,PYH20210331118100013,20210331,03,...,image/jpeg,P4,PYH2021033111810001300_P4.jpg,PREVIEW,KR,전국 농민 투쟁선포 기자회견,전국 농민 투쟁선포 기자회견\r\n (서울=연합뉴스) 윤동진 기자 = 31일 ...,2021-03-31T13:02:42,NaN,NaN
6,rens_test,_doc,PYH20210331118200013,1.0,130901,C,PYH0,PYH20210331118200013,20210331,03,...,image/jpeg,P4,PYH2021033111820001300_P4.jpg,PREVIEW,KR,전국 농민 투쟁선포 기자회견,전국 농민 투쟁선포 기자회견\r\n (서울=연합뉴스) 윤동진 기자 = 31일 ...,2021-03-31T13:09:01,NaN,NaN
7,rens_test,_doc,PYH20210331118300013,1.0,130903,C,PYH0,PYH20210331118300013,20210331,03,...,image/jpeg,P4,PYH2021033111830001300_P4.jpg,PREVIEW,KR,전국 농민 투쟁선포 기자회견,전국 농민 투쟁선포 기자회견\r\n (서울=연합뉴스) 윤동진 기자 = 31일 ...,2021-03-31T13:09:03,NaN,NaN
8,rens_test,_doc,PYH20210331118400013,1.0,130909,C,PYH0,PYH20210331118400013,20210331,NaN,...,image/jpeg,P4,PYH2021033111840001300_P4.jpg,PREVIEW,KR,'공식 사죄 언제쯤','공식 사죄 언제쯤'\r\n (서울=연합뉴스) 이지은 기자 = 31일 오후 서...,2021-03-31T13:09:09,"[{'code': '03', 'name': '사회'}, {'code': '04', ...","[{'code': '0501001', 'name': '사회일반'}, {'code':..."
9,rens_test,_doc,PYH20210331118500013,1.0,130912,C,PYH0,PYH20210331118500013,20210331,NaN,...,image/jpeg,P4,PYH2021033111850001300_P4.jpg,PREVIEW,KR,'계속되는 수요시위','계속되는 수요시위'\r\n (서울=연합뉴스) 이지은 기자 = 31일 오후 서...,2021-03-31T13:09:12,"[{'code': '03', 'name': '사회'}, {'code': '04', ...","[{'code': '0501001', 'name': '사회일반'}, {'code':..."


In [21]:
index = 'rens_test'

def make_body(start, size=1000):
    #쿼리 정의 (전체 데이터 불러오기) 
    body= {
        "track_total_hits" : True,
        "from": start,
        "size": start+size, 
        "query": {"match_all": { }
        }}
    return body

In [22]:
size = 1000
#Data 전체 수 가져오기
total = es.search(index= index, body=make_body(start=0))['hits']['total']['value']
#Data 형태 보기
total1 = es.search(index= index, body=make_body(start=0))

In [23]:
total

952545

In [24]:
total1

{'took': 13,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 952545, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'rens_test',
    '_type': '_doc',
    '_id': 'PYH20210331117600013',
    '_score': 1.0,
    '_source': {'YNewsML': {'Header': {'SendTime': 130204,
       'Action': 'C',
       'ServiceType': 'PYH0',
       'ContentID': 'PYH20210331117600013',
       'SendDate': 20210331},
      'Metadata': {'Category': {'code': '03', 'name': '사회'},
       'Credit': 'YNA',
       'PublishNo': '2442',
       'Region': {'RegionCode': {'code': 7002001, 'name': '서울'}},
       'Desk': {'code': '013', 'content': '사진부'},
       'Href': 'https://www.yna.co.kr/view/PYH20210331117600013?section=search',
       'Class': {'ClassCode': {'code': '0501001', 'name': '사회일반'}},
       'Urgency': 3,
       'Source': 'YNA',
       'Writer': {'uid': 'null', 'content': '윤동진'}},
      'NewsContent': {'TaggedBody': '여의서로 벚꽃길 오늘붜

### ES데이터 뽑아보기

In [25]:
text = '이태원'

In [31]:
size=10000
def make_body(start, size=10000):
    body= {
        "track_total_hits" : True,
        "from": start,
        "size": start+size, 
        "_source": [
            "YNewsML.Metadata.Category.name",
            "YNewsML.NewsContent.Title",
            "YNewsML.NewsContent.Body",
            "YNewsML.Metadata.Href",
            "send_timestamp"
        ], 
        "sort" : {
            "send_timestamp" : "desc"
        },
        "query": {
            "bool":{
              "filter": [
                {
                 "match_phrase": {
                   "YNewsML.NewsContent.Title": text
                  }
                }
              ]
            }
        }
    }
    return body

In [32]:
#Data 전체 수 가져오기
total = es.search(index= index, body=make_body(start=0))['hits']['total']['value']
#Data 형태 보기
total1 = es.search(index= index, body=make_body(start=0))

In [33]:
total

4373

In [34]:
## 잘 뽑히나 테스트
#데이터 불러오기
result = es.search(index= index, body=make_body(start=1, size=1000), request_timeout=6000)
#데이터 type이 dict이므로 dataframe으로 변환, df에 저장
df = Select.from_dict(result).to_pandas()
df

,_index,_type,_id,_score,YNewsML.Metadata.Category.name,YNewsML.Metadata.Href,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,send_timestamp,YNewsML.Metadata.Category
0,rens_test,_doc,PYH20221230129000013,None,사회,https://www.yna.co.kr/view/PYH2022123012900001...,10·29 이태원 참사 2차 시민추모제,10·29 이태원 참사 2차 시민추모제\r\n (서울=연합뉴스) 이지은 기자 ...,2022-12-30T19:30:56,NaN
1,rens_test,_doc,AKR20221230091800004,None,사회,https://www.yna.co.kr/view/AKR2022123009180000...,"이태원 참사 유족, '분향소 시위' 보수단체 접근금지 신청","이태원 참사 유족, '분향소 시위' 보수단체 접근금지 신청\n\r\n<YNAOBJE...",2022-12-30T15:03:58,NaN
2,rens_test,_doc,AKR20221230030800004,None,사회,https://www.yna.co.kr/view/AKR2022123003080000...,'이태원 참사' 이임재 전 용산경찰서장 구속 송치,'이태원 참사' 이임재 전 용산경찰서장 구속 송치\n업무상과실치사상 등 혐의…송병주...,2022-12-30T10:30:38,NaN
3,rens_test,_doc,AKR20221229126652001,None,정치,https://www.yna.co.kr/view/AKR2022122912665200...,"이태원 국조…""경찰은 할 말 없다"" ""서울시, 파장 축소"" 질타(종합2보)","이태원 국조…""경찰은 할 말 없다"" ""서울시, 파장 축소"" 질타(종합2보)\n두번째...",2022-12-29T22:48:39,NaN
4,rens_test,_doc,AKR20221229081251004,None,사회,https://www.yna.co.kr/view/AKR2022122908125100...,"대검 ""이태원 유족에 '마약 부검' 제안 안해""(종합)","대검 ""이태원 유족에 '마약 부검' 제안 안해""(종합)\n국조특위 기관보고…""유족에...",2022-12-29T21:07:29,NaN
...,...,...,...,...,...,...,...,...,...,...
996,rens_test,_doc,AKR20221115084700017,None,NaN,https://www.yna.co.kr/view/AKR2022111508470001...,"방심위, ""이태원 과거엔 일방통행"" 김어준 발언 놓고 논란","방심위, ""이태원 과거엔 일방통행"" 김어준 발언 놓고 논란\n김어준 민감발언 3건이...",2022-11-15T14:27:33,"[{'name': '경제'}, {'name': 'IT/과학'}]"
997,rens_test,_doc,AKR20221115090300004,None,사회,https://www.yna.co.kr/view/AKR2022111509030000...,"이태원 희생자 명단 공개, 법적 문제 없나","이태원 희생자 명단 공개, 법적 문제 없나\n법조계 ""개인정보법·사자명예훼손 위반 ...",2022-11-15T13:43:54,NaN
998,rens_test,_doc,PYH20221115093100013,None,정치,https://www.yna.co.kr/view/PYH2022111509310001...,'이태원 참사 국정조사 촉구','이태원 참사 국정조사 촉구'\r\n (서울=연합뉴스) 백승렬 기자 = 15일...,2022-11-15T12:40:22,NaN
999,rens_test,_doc,PYH20221115093000013,None,정치,https://www.yna.co.kr/view/PYH2022111509300001...,'이태원 참사 국정조사 촉구','이태원 참사 국정조사 촉구'\r\n (서울=연합뉴스) 백승렬 기자 = 15일...,2022-11-15T12:40:20,NaN


In [38]:
df['YNewsML.Metadata.Href'][1]

'https://www.yna.co.kr/view/AKR20221230091800004?input=2508m'

In [226]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   _index                          97 non-null     object
 1   _type                           97 non-null     object
 2   _id                             97 non-null     object
 3   _score                          0 non-null      object
 4   YNewsML.Metadata.Category.name  76 non-null     object
 5   YNewsML.NewsContent.Title       97 non-null     object
 6   YNewsML.NewsContent.Body        97 non-null     object
 7   send_timestamp                  97 non-null     object
 8   YNewsML.Metadata.Category       21 non-null     object
dtypes: object(9)
memory usage: 6.9+ KB


In [227]:
df['send_timestamp'] = pd.to_datetime(df['send_timestamp'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   _index                          97 non-null     object        
 1   _type                           97 non-null     object        
 2   _id                             97 non-null     object        
 3   _score                          0 non-null      object        
 4   YNewsML.Metadata.Category.name  76 non-null     object        
 5   YNewsML.NewsContent.Title       97 non-null     object        
 6   YNewsML.NewsContent.Body        97 non-null     object        
 7   send_timestamp                  97 non-null     datetime64[ns]
 8   YNewsML.Metadata.Category       21 non-null     object        
dtypes: datetime64[ns](1), object(8)
memory usage: 6.9+ KB


In [228]:
df_add = df[(df.send_timestamp.dt.month == 10) & (df.send_timestamp.dt.year == 2022)]
df_add

,_index,_type,_id,_score,YNewsML.Metadata.Category.name,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,send_timestamp,YNewsML.Metadata.Category
18,rens_test,_doc,PYH20221027121600013,None,정치,민주당 금융위기 대책 긴급현장점검회의,민주당 금융위기 대책 긴급현장점검회의 \r\n (서울=연합뉴스) 더불어민주당 ...,2022-10-27 14:15:56,NaN
19,rens_test,_doc,AKR20221020059700001,None,정치,與 싱크탱크 여의도연구원 원장에 3선 출신 김용태 전 의원,與 싱크탱크 여의도연구원 원장에 3선 출신 김용태 전 의원\n\r\n (서울=...,2022-10-20 10:32:41,NaN
20,rens_test,_doc,AKR20221007059100002,None,경제,새출발기금 출범 3일만에 채무조정 신청액 1조원 넘어,새출발기금 출범 3일만에 채무조정 신청액 1조원 넘어\n사전접수 기간 포함 7일간 ...,2022-10-07 10:35:34,NaN
21,rens_test,_doc,AKR20221006065500002,None,경제,새출발기금에 6일간 5천903명 채무조정 신청…9천500억원 규모,새출발기금에 6일간 5천903명 채무조정 신청…9천500억원 규모\n\r\n\r\n...,2022-10-06 10:44:44,NaN
22,rens_test,_doc,AKR20221004023251002,None,경제,취약 자영업자에 30조원 맞춤형 채무조정…'새출발기금' 출범(종합),취약 자영업자에 30조원 맞춤형 채무조정…'새출발기금' 출범(종합)\n온·오프라인 ...,2022-10-04 13:42:45,NaN
23,rens_test,_doc,PYH20221004027200013,None,경제,새출발기금 출범,새출발기금 출범\r\n (서울=연합뉴스) 한상균 기자 = 4일 서울 캠코(한국...,2022-10-04 09:38:48,NaN
24,rens_test,_doc,PYH20221004027100013,None,경제,새출발기금 출범,새출발기금 출범\r\n (서울=연합뉴스) 한상균 기자 = 4일 서울 캠코(한국...,2022-10-04 09:38:36,NaN
25,rens_test,_doc,PYH20221004027000013,None,경제,소상공인을 위한 새출발기금 출범,소상공인을 위한 새출발기금 출범\r\n (서울=연합뉴스) 한상균 기자 = 4일...,2022-10-04 09:38:32,NaN
26,rens_test,_doc,PYH20221004026600013,None,경제,소상공인을 위한 새출발기금 출범,소상공인을 위한 새출발기금 출범\r\n (서울=연합뉴스) 한상균 기자 = 4일...,2022-10-04 09:38:13,NaN
27,rens_test,_doc,PYH20221004026500013,None,경제,소상공인을 위한 새출발기금 출범,소상공인을 위한 새출발기금 출범\r\n (서울=연합뉴스) 한상균 기자 = 4일...,2022-10-04 09:36:43,NaN


In [229]:
df_add['day'] = df_add.send_timestamp.dt.day
df_add = df_add[df_add.day >= 20]
pv_add = pd.pivot_table(df_add, index='day', values='YNewsML.NewsContent.Title', aggfunc='count')
pv_add = pv_add.reset_index()
pv_add

<ipython-input-229-bbbf69f3e23d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_add['day'] = df_add.send_timestamp.dt.day


,day,YNewsML.NewsContent.Title
0,20,1
1,27,1


In [230]:
pv_add.columns = ['day', text]
pv_add

,day,국회 정무위원장
0,20,1
1,27,1


In [231]:
df_concat = pd.merge(df_concat, pv_add, how="outer", on="day").sort_values('day')
df_concat


,day,러시아군의 공격을,여행 수요가,서울디지털재단,ETF,핼러윈,원전,억원작년 동기,김태종 특파원,대한민국시도의회,행정안전부 장관이,폭발,지진,국정원장,국회 정무위원장
0,20,1.0,NaN,NaN,2.0,NaN,11.0,11.0,1.0,1.0,17.0,4.0,2.0,NaN,1.0
1,21,3.0,NaN,NaN,2.0,NaN,11.0,11.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN
2,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN
3,23,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24,NaN,1.0,NaN,1.0,NaN,6.0,6.0,4.0,NaN,6.0,1.0,1.0,NaN,NaN
5,25,NaN,NaN,NaN,3.0,NaN,11.0,11.0,2.0,NaN,3.0,2.0,1.0,NaN,NaN
6,26,1.0,NaN,NaN,NaN,NaN,7.0,7.0,2.0,NaN,2.0,1.0,4.0,26.0,NaN
7,27,NaN,1.0,NaN,5.0,NaN,10.0,10.0,1.0,NaN,NaN,3.0,1.0,21.0,1.0
8,28,NaN,NaN,NaN,2.0,NaN,4.0,4.0,2.0,NaN,6.0,2.0,NaN,2.0,NaN
9,29,NaN,NaN,NaN,NaN,NaN,7.0,7.0,NaN,NaN,5.0,NaN,67.0,NaN,NaN


In [234]:
df_concat.to_csv('key.csv',index=False)

In [26]:
# 12월 1일, 주요 카테고리 순위 
df['YNewsML.Metadata.Category.name'].value_counts() 

국제    60
경제     4
정치     2
기타     1
사회     1
Name: YNewsML.Metadata.Category.name, dtype: int64

In [27]:
index = 'rens_test'
size = 1000
df = pd.DataFrame()

for i in tqdm(range(0, total+size, size)):
    #데이터 불러오기
    result = es.search(index= index, body=make_body(start=i, size=size), request_timeout=600000)
    #데이터 type이 dict이므로 dataframe으로 변환, df에 저장
    df_result = Select.from_dict(result).to_pandas()
    df = pd.concat([df,df_result])

100%|██████████| 2/2 [00:00<00:00, 92.55it/s]


In [28]:
df.tail(2)

,_index,_type,_id,_score,YNewsML.Metadata.Category.name,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,send_timestamp
67,rens_test,_doc,AKR20220224196500009,None,국제,"[우크라 침공] 심리전 치열…러·우크라, 언론에 앞다퉈 전적 과시","[우크라 침공] 심리전 치열…러·우크라, 언론에 앞다퉈 전적 과시\n우크라 ""러시아...",2022-02-24T21:35:17
68,rens_test,_doc,AKR20220224117300009,None,국제,"[속보] 러 ""우크라 군시설 정밀 타격…민간인에 위협 없어""","[속보] 러 ""우크라 군시설 정밀 타격…민간인에 위협 없어""\n\r\n<YNAOBJ...",2022-02-24T13:45:38


In [29]:
# df.to_csv('news220907.csv', index=False)

### 데이터 조작

In [ ]:
df.columns

## 1차 분석

In [ ]:
body= {
  "query": {"match_all": {
  
}}}
result = es.search(index='rens_test', body=body, request_timeout=30)

In [ ]:
Select.from_dict(result).to_pandas()

In [ ]:
def make_body(start, size=1000):
    #쿼리 정의 (전체 데이터 불러오기) 
    body= {
        "track_total_hits" : True,
        "from": start,
        "size": start+size, 
        "query": {
      "bool":{
      "filter": [{"range": {
      "send_timestamp": {
        "gte": "2021-08-11T00:00:00.000Z",
        "format": "strict_date_optional_time"
      }}
    },{"match": {
         "YNewsML.NewsContent.Body": {
            "query":"소방 119 구조 화재 구급 원인 교통사고 승강기 자살기도 산악사고 수해 추락 끼임 붕괴 도괴 폭발 누출 테러 항공기사고 벌집제거 고혈압 당뇨 뇌혈관질환 심장질환 폐질환 결핵 간염 간경화 알레르기 암 신부전 낙상 둔상 열상 자상 관통상 기계 농기계 중독 연기흡입 질식 기도막힘 화상 상해 두통 흉통 복통 성폭행 분만진통 골절 탈구 삠 찰과상 타박상 절단 압궤손상 화상 의식장애 호흡곤란 기침 호흡정지 심정지 경련 발작 실신 구토 설사 객혈 토혈 고열 저체온증 어지러움 마비 전신쇠약 산불 지하철 선박사고 가스사고 원전사고 호우 대설 강풍 태풍 우박 낙뢰 한파 풍랑 지진 폭염 가뭄 홍수 황사 침수 해일 화산폭발 산사태 여름철물놀이 응급처치 해파리 심폐소생술 놀이시설 갇힘 임산부 철도사고 산업재해 붉은불개미 식중독 자전거사고 레저사고 인명사고 주취 담뱃불 불티 스파크 용접 라이터불 성냥불 낙뢰 누전 담배꽁초 불장난 실화 쓰레기 소각 난로 연기흡입 과열 방화 발화 배선 온풍기 부주의 합선 가연성 병충해 자살 재난 안전 전염 산악 유류 실족 가스레인지 발열 선박 압사 감염 동파 한랭질환자 학대 바이러스 고압 전류 브레이크잠김방지시스템 결함 담뱃갑 폭우 방임 제동장치 학대 야생보호 위험물 몸살 누수 기저질환 과다복용 유출 충격 독극물 액화석유가스 폐렴 감전 익사 해안사고 피탄지 강추위 전복 배터리 낙석 비 미열 다이너마이트 금속화재 빗길 악천후 근육통 음주운전 과로사 적조 녹조 고장 참사 쇼크 방사능 폭설 불씨 뇌색전 수난사고 하론 무단횡단 건조 블랙아이스 빙판길 뇌졸중 불발탄 폭풍 고립 난파 페놀 범람 추돌 경상 중상 부상 쓰나미 조난 화염 쏘임 화마 미끄러짐 넘어짐 시너 극단선택 리튬 메탄 탱크로리 여진 결빙 트리클로로메탄 강진 토치 소방드론 촛불 뇌경색 졸음운전 부정맥 탈진 산악실종 알킬레이트 수질사고 낙하물 폭죽 출혈 열사병 필로폰 연탄가스 열돔 장마 일사병 사이안화수소 투신 싱크홀 포트홀 자일렌 황화수소 맨홀 철거 아세톤 LNG 전기적 전기 사망 후송 매몰 탈선 진화 염산 메탄올 취사",
            "operator":"OR"
        }
        }}]
  }}}
    return body

In [ ]:
index = 'rens_test'
size = 1000
#Data 전체 수 가져오기
total = es.search(index= index, body=make_body(start=0))['hits']['total']['value']
#Data 형태 보기
total1 = es.search(index= index, body=make_body(start=0))

In [ ]:
total

In [ ]:
size = 10000
index ='rens_test'
df = pd.DataFrame()
for i in tqdm(range(0, total+size, size)):
        result = es.search(index= index, sort='send_timestamp', body=make_body(start=i, size=size), request_timeout=30)
        result1 = Select.from_dict(result).to_pandas()
        df = pd.concat([df,result1])

In [ ]:
df1 = df.reset_index().drop("index",axis=1)

In [ ]:
df1.columns

In [ ]:
type(df1)

In [ ]:
df2 = df1.drop_duplicates(subset='YNewsML.NewsContent.Body')

In [ ]:
df2

In [ ]:
df2 = df2.reset_index().drop('index',axis=1)

In [ ]:
a = []

for i in df2['YNewsML.NewsContent.Title']:
    if '과천 방음터널' in i:
        a.append(i)
    elif '경인고속도로' in i:
        a.append(i)
    elif '방음터널' in i:
        a.append(i)
    elif '제2경인고속도' in i:
        a.append(i)  

In [ ]:
cond1 = df2['YNewsML.NewsContent.Title'].isin(a)
df313 = pd.DataFrame(df2.loc[cond1]).reset_index().drop('index',axis=1)

In [ ]:
df313 = df313[['_id','Keyword','NamedEntity']]

In [ ]:
def cleansing(body, mode=None):
    if type(body) == np.ndarray:
        body = np.round(body.tolist(), 5)
        body = body.tolist()
        return body
    else:
        body = re.sub('<YNAOBJECT.*?/YNAOBJECT>', '', body, 0, re.I|re.S) # YNAOBJECT 태그 제거
        body = re.sub('<table.*?/table>', '', body, 0, re.I|re.S) # table 태그 제거
        
        if mode == 'meta':
            try:
                body = body[re.search("\(.*?연합뉴스\).*?=", body).span()[0]:]
            except AttributeError:
                pass
            return body
        
        if mode == 'sum':
            try:
                body = body[re.search("\(.*?연합뉴스\).*?=", body).span()[1]:]
            except AttributeError:
                pass
            
        
        body = re.sub(r'\([^)]*\)', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub(r'\[.*\]|\s-\s.*', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub(r'\<.*\>|\s-\s.*', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub('·', ' ', body, 0, re.I|re.S) # 변환
        body = re.sub('-', ' ', body, 0, re.I|re.S) # 변환
        body = re.sub('\(.*?연합뉴스\).*?=', '', body, 0, re.I|re.S) # 기자명 및 기자소속 제거
        body = re.sub('\(서울.*?연합뉴스\)', '', body, 0, re.I|re.S) # 기자소속 제거
        body = re.sub('yna.co.kr', '', body, 0, re.I|re.S) # 연합 url 제거
        body = re.sub('([a-zA-Z0-9\_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', '', body, 0, re.I|re.S) # E-mail 제거
        body = re.sub('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', body, 0, re.I|re.S) # URL 제거
#         body = re.sub('([0-9\_.+-]+억[0-9-]+천[0-9-]+만원)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+천[0-9-]+억원)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+억[0-9-]+천[0-9-]+달러)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+억[0-9-]+천만달러)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+천[0-9-]+만원)', '', body, 0, re.I|re.S) # 천만 제거
#         body = re.sub('([0-9\_.+-]+천[0-9-]+달러)', '', body, 0, re.I|re.S) # 천만 제거
#         body = re.sub('([0-9\_.+-]+만원)', '', body, 0, re.I|re.S) # 만 제거
#         body = re.sub('([0-9\_.+-]+만[0-9-]천[0-9-]+원)', '', body, 0, re.I|re.S) # 만 제거
        body = re.sub('전문보기.*?\n', '', body, 0, re.I|re.S) # 전문보기 제거
        body = re.sub('&lt;.*?&gt;&gt;', '', body, 0, re.I|re.S) # 기타 안내문 제거
        body = re.sub('&lt;.*?&gt;', '', body, 0, re.I|re.S) # '&gt;'가 1개인 안내문 제거
        body = re.sub('&apos;', "'", body, 0, re.I|re.S) # &apos; ' 변환
        body = re.sub('&amp;', "&", body, 0, re.I|re.S) # &amp;; ' 변환        
        body = re.sub('-.*?_.*?\(끝\)', '', body, 0, re.I|re.S) # 9001887 속성에만 존재하는 불용어 제거
        body = re.sub('[#@*※|》■▲▼◀▶◇☎]-<>=+·↑', '', body, 0, re.I|re.S) # 특수문자 제거
        body = re.sub('…', ' ', body, 0, re.I|re.S) # … 제거
        body = re.sub('\(끝\)', '', body, 0, re.I|re.S) # 끝 태그 제거
        body = re.sub('[""]', '', body, 0, re.I|re.S) # "" 제거
#         body = re.sub(r'[^ A-Za-z가-힣+]',' ' , body, 0, re.I|re.S) # 한글, 알파벳을 제외한 나머지 제거


        paragraphs = body.split('\r\n') # 단락 분리
        if ' 기자' in paragraphs[-1]: del paragraphs[-1] # 끝 단락 기자명 제거
        body = []
        for paragraph in paragraphs:
            paragraph = re.sub('\[.+?\]', '', paragraph, 0, re.I|re.S)
            paragraph = ' '.join(paragraph.split()) # 문자열 중간 다중 공백 제거
            if len(paragraph) > 0:
                body.append(paragraph)
        
        if mode == 'quot':
            return body
        else:
            return ' '.join(body)

In [ ]:
df313['ISSUE_ID'] = np.nan

In [ ]:
df2 = df313.rename(columns={'_id':'NEWS_ID','YNewsML.NewsContent.Title':'TITLE','send_timestamp':'TIMESTAMP','YNewsML.NewsContent.Body':'BODY','YNewsML.NewsContent.MultiMedia.DataFile.content':'PICTURE','YNewsML.Metadata.Href':'URL','YNewsML.Metadata.Class.ClassCode':'CATEGORY_NAME','YNewsML.Metadata.Region.RegionCode.name':'REGION_NAME','YNewsML.NewsContent.LangType':'NATION'})

In [ ]:
df2 = df313.rename(columns={'_id':'NEWS_ID',"Keyword":"KEYWORD_NAME","NamedEntity":"NER_WORD"}).drop("ISSUE_ID",axis=1)

In [ ]:
df2 = df2.dropna().reset_index().drop('index',axis=1)

In [ ]:
EV_word_all = []
for i in range(len(df2)):
    try:
        for j in range(len(df2['NER_WORD'][i])):
                EV_word_all.append([df2['NEWS_ID'][i],df2['NER_WORD'][i][j]['code'],df2['NER_WORD'][i][j]['word']])
    except:
        EV_word_all.append([df2['NEWS_ID'][i],np.nan])

In [ ]:
pd.DataFrame(EV_word_all).rename(columns={0:'NEWS_ID',1:"NER_CODE",2:"NER_WORD"}).to_csv('HIGHWAY_NEWS_NER.csv',encoding='utf-8-sig',index=False)

In [ ]:
EV_word_all = []
for i in range(len(df2)):
    try:
        for j in range(len(df2['KEYWORD_NAME'][i])):
                EV_word_all.append([df2['NEWS_ID'][i],df2['KEYWORD_NAME'][i][j]])
    except:
        EV_word_all.append([df2['NEWS_ID'][i],np.nan])

In [ ]:
pd.DataFrame(EV_word_all).rename(columns = {0:'NEWS_ID',1:'KEYWORD_NAME'}).to_csv('HIGHWAY_RELATED_KEYWORD.csv',encoding='utf-8-sig',index=False)

In [ ]:
df2['BODY'] = df2['BODY'].apply(cleansing)

In [ ]:
df2

In [ ]:
df2

In [ ]:
df2.to_csv('HIGHWAY_MAINTABLE.csv',encoding='utf-8-sig',index=False)

In [ ]:
all = df313[['_id','YNewsML.Metadata.Class.ClassCode']].dropna().reset_index().drop('index',axis=1)

In [ ]:
EV_word_all = []
for i in range(len(all)):
    try:
        for j in range(len(all['YNewsML.Metadata.Class.ClassCode'][i])):
                EV_word_all.append([all['_id'][i],all['YNewsML.Metadata.Class.ClassCode'][i][j]['code']])
    except:
        EV_word_all.append([all['_id'][i],np.nan])

In [ ]:
pd.DataFrame(EV_word_all).rename(columns={0:'NEWS_ID',1:'ACCIDENT_INDEX'}).to_csv('HIGHWAY_NEWS_CATEGORY.csv',encoding='utf-8-sig',index=False)

In [ ]:
df2.to_csv('fire_news.csv',encoding='utf-8-sig',index=False)

In [ ]:
df2.columns.tolist()

In [ ]:
df3 = df313[['_id','YNewsML.NewsContent.Title','YNewsML.NewsContent.Body','Keyword']].dropna(subset='Keyword')

In [ ]:
df3

In [ ]:
a = []

for i in df2['YNewsML.NewsContent.Title']:
    if '이태원' in i:
        a.append(i)

In [ ]:
cond1 = df2['YNewsML.NewsContent.Title'].isin(a)

In [ ]:
df313 = pd.DataFrame(df2.loc[cond1]).reset_index().drop('index',axis=1)

In [ ]:
df3 = df3.reset_index().drop('index',axis=1)

In [ ]:
EV_word_all = []
for i in range(len(df3)):
    try:
        for j in range(len(df3['NamedEntity'][i])):
                EV_word_all.append([df3['_id'][i],df3['NamedEntity'][i][j]['code'],df3['NamedEntity'][i][j]['word']])
    except:
        EV_word_all.append([df3['_id'][i],np.nan])

In [ ]:
pd.DataFrame(EV_word_all).rename(columns={0:'NEW_ID',1:'NER_CODE',2:'NER_WORD'}).to_csv('NEWS_NER.csv',encoding='utf-8-sig',index=False)

In [ ]:
EV_word_all = []
for i in range(len(df3)):
    try:
        for j in range(len(df3['Keyword'][i])):
                EV_word_all.append([df3['_id'][i],df3['Keyword'][i][j]])
    except:
        EV_word_all.append([df3['_id'][i],np.nan])

In [ ]:
pd.DataFrame(EV_word_all).rename(columns={0:'NEWS_ID',1:'KEYWORD_NAME'}).to_csv('RELATED_KEYWORD.csv',encoding='utf-8-sig',index=False)

In [ ]:
df313.to_csv('이태원.csv',encoding='utf-8-sig',index=False)

In [ ]:
df1_NER = pd.DataFrame(df3.loc[cond1]['NamedEntity'].dropna()).reset_index().drop('index',axis=1)

In [ ]:
df313['YNewsML.Metadata.Class.ClassCode'] = df313['YNewsML.Metadata.Class.ClassCode'].fillna(0)

In [ ]:
df313.columns

In [ ]:
all = df313[['_id','YNewsML.Metadata.Class.ClassCode']].dropna().reset_index().drop('index',axis=1)

In [ ]:
all

In [ ]:
EV_word_all = []
for i in range(len(all)):
    try:
        for j in range(len(all['YNewsML.Metadata.Class.ClassCode'][i])):
                EV_word_all.append([all['_id'][i],all['YNewsML.Metadata.Class.ClassCode'][i][j]['code']])
    except:
        EV_word_all.append([all['_id'][i],np.nan])

In [ ]:
pd.DataFrame(EV_word_all).rename(columns={0:'NEWS_ID',1:'ACCIDENT_INDEX'}).to_csv('NEWS_CATEGORY.csv',encoding='utf-8-sig',index=False)

In [ ]:
pd.DataFrame(EV_word_all).value_counts()

In [ ]:
pd.DataFrame(EV_word_all).value_counts().index.tolist()

In [ ]:
df4 = pd.DataFrame(df1_NER['NamedEntity'].dropna()).reset_index().drop('index',axis=1)

In [ ]:
df4['NamedEntity'][0]

In [ ]:
EV_word = []
for i in range(len(df4)):
    for j in range(len(df4['NamedEntity'][i])):
        if df4['NamedEntity'][i][j]['code'] == 'EV':
            EV_word.append(df4['NamedEntity'][i][j]['word'])

In [ ]:
pd.DataFrame([pd.DataFrame(EV_word).value_counts().index.to_list(),pd.DataFrame(EV_word).value_counts().tolist()]).T.rename(columns={0:'키워드',1:'counts'}).to_csv('클래스코드적용전.csv',encoding='utf-8-sig',index=False)

In [ ]:
pd.DataFrame(pd.DataFrame(EV_word).value_counts()).to_csv('클래스코드적용전.csv',encoding='utf-8-sig',index=False)

In [ ]:
def make_body(start, size=1000):
    #쿼리 정의 (전체 데이터 불러오기) 
    body= {
        "track_total_hits" : True,
        "from": start,
        "size": start+size, 
        "query": {
    "bool":{
        "must": [
          {"match": {
         "YNewsML.NewsContent.Body": {
            "query":"소방 119 구조 화재 구급 원인 교통사고 승강기 자살기도 산악사고 수해 추락 끼임 붕괴 도괴 폭발 누출 테러 항공기사고 벌집제거 고혈압 당뇨 뇌혈관질환 심장질환 폐질환 결핵 간염 간경화 알레르기 암 신부전 낙상 둔상 열상 자상 관통상 기계 농기계 중독 연기흡입 질식 기도막힘 화상 상해 두통 흉통 복통 성폭행 분만진통 골절 탈구 삠 찰과상 타박상 절단 압궤손상 화상 의식장애 호흡곤란 기침 호흡정지 심정지 경련 발작 실신 구토 설사 객혈 토혈 고열 저체온증 어지러움 마비 전신쇠약 산불 지하철 선박사고 가스사고 원전사고 호우 대설 강풍 태풍 우박 낙뢰 한파 풍랑 지진 폭염 가뭄 홍수 황사 침수 해일 화산폭발 산사태 여름철물놀이 응급처치 해파리 심폐소생술 놀이시설 갇힘 임산부 철도사고 산업재해 붉은불개미 식중독 자전거사고 레저사고 인명사고 주취 담뱃불 불티 스파크 용접 라이터불 성냥불 낙뢰 누전 담배꽁초 불장난 실화 쓰레기 소각 난로 연기흡입 과열 방화 발화 배선 온풍기 부주의 합선 가연성 병충해 자살 재난 안전 전염 산악 불 유류 실족 가스레인지 발열 선박 가스 압사 감염 동파 한랭질환자 학대 전기 바이러스 고압 전류 브레이크잠김방지시스템 결함 담뱃갑 폭우 방임 제동장치 학대 야생보호 위험물 몸살 누수 기저질환 과다복용 유출 충격 독극물 액화석유가스 폐렴 감전 익사 해안사고 피탄지 강추위 전복 배터리 낙석 비 미열 다이너마이트 금속화재 빗길 악천후 근육통 음주운전 과로사 적조 녹조 고장 참사 쇼크 방사능 폭설 불씨 뇌색전 수난사고 하론 무단횡단 건조 블랙아이스 빙판길 뇌졸중 불발탄 폭풍 고립 난파 페놀 범람 추돌 경상 중상 부상 쓰나미 조난 화염 쏘임 화마 미끄러짐 넘어짐 시너 극단선택 리튬 메탄 탱크로리 여진 결빙 트리클로로메탄 강진 토치 소방드론 촛불 뇌경색 졸음운전 부정맥 탈진 산악실종 알킬레이트 수질사고 낙하물 폭죽 출혈 열사병 필로폰 연탄가스 열돔 장마 일사병 사이안화수소 투신 싱크홀 포트홀 자일렌 황화수소 맨홀 철거 아세톤 LNG",
            "operator":"OR"
        }}}
        ], 
        "must_not": [
           {"match_phrase": {
         "YNewsML.Metadata.Desk.content": "사진부"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "0801002"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "0402004"
        }}],
        "filter": [{"range": {
      "send_timestamp": {
        "gte": "2021-08-11T00:00:00.000Z",
        "format": "strict_date_optional_time"
      }}
    },{"terms": {
          "YNewsML.Metadata.Class.ClassCode.code": [
            "606001",
            "606002",
            "606003",
            "606004",
            "606005",
            "606006",
            "606007",
            "606008",
            "606009",
            "606010",
            "606011",
            "606012",
            "606013",
            "607001",
            "607002",
            "607003",
            "607004",
            "607005",
            "607006",
            "0501001",
            "0510004",
            "0501018",
            "0501004",
            "0505005",
            "0506001",
            "0506002",
            "0506003",
            "0506004",
            "0506005",
            "0507010",
            "0507009",
            "0507007",
            "0101001",
            "0103003",
            "0103004",
            "0103005",
            "0103006",
            "0103007",
            "0103008",
            "0103002",
            "0106001",
            "0106003",
            "0106005",
            "0106006",
            "0107001",
            "0107002",
            "0107004",
            "0107005",
            "0901001",
            "1101001",
            "1212001",
            "1210001",
            "1207001",
            "1201001"
          ]
        }}]
  }}}
    return body

In [ ]:
size = 10000
index ='rens_test'
df = pd.DataFrame()
for i in tqdm(range(0, total+size, size)):
        result = es.search(index= index, sort='send_timestamp', body=make_body(start=i, size=size), request_timeout=30)
        result1 = Select.from_dict(result).to_pandas()
        df = pd.concat([df,result1])

In [ ]:
df5 = df[['YNewsML.NewsContent.Title','YNewsML.NewsContent.Body','NamedEntity']]

In [ ]:
all1 = pd.DataFrame(df5.NamedEntity.dropna()).reset_index().drop('index',axis=1)

In [ ]:
all1

In [ ]:
EV_word_all1 = []
for i in range(len(all1)):
    for j in range(len(all1['NamedEntity'][i])):
        if all1['NamedEntity'][i][j]['code'] == 'EV':
            EV_word_all1.append(all1['NamedEntity'][i][j]['word'])

In [ ]:
pd.DataFrame(EV_word_all1).value_counts()

In [ ]:
EV_word_all1_list = pd.DataFrame([pd.DataFrame(EV_word_all1).value_counts().index.to_list(),pd.DataFrame(EV_word_all1).value_counts().to_list()]).T
EV_word_all1_list

In [ ]:
EV_word_all1_list.to_csv('클래스코드적용후_manyclass.csv',encoding='utf-8-sig',index=False)

In [ ]:
def make_body(start, size=1000):
    #쿼리 정의 (전체 데이터 불러오기) 
    body= {
        "track_total_hits" : True,
        "from": start,
        "size": start+size, 
        "query": {
    "bool":{
        "must": [
          {"match": {
         "YNewsML.NewsContent.Body": {
            "query":"소방 119 구조 화재 구급 원인 교통사고 승강기 자살기도 산악사고 수해 추락 끼임 붕괴 도괴 폭발 누출 테러 항공기사고 벌집제거 고혈압 당뇨 뇌혈관질환 심장질환 폐질환 결핵 간염 간경화 알레르기 암 신부전 낙상 둔상 열상 자상 관통상 기계 농기계 중독 연기흡입 질식 기도막힘 화상 상해 두통 흉통 복통 성폭행 분만진통 골절 탈구 삠 찰과상 타박상 절단 압궤손상 화상 의식장애 호흡곤란 기침 호흡정지 심정지 경련 발작 실신 구토 설사 객혈 토혈 고열 저체온증 어지러움 마비 전신쇠약 산불 지하철 선박사고 가스사고 원전사고 호우 대설 강풍 태풍 우박 낙뢰 한파 풍랑 지진 폭염 가뭄 홍수 황사 침수 해일 화산폭발 산사태 여름철물놀이 응급처치 해파리 심폐소생술 놀이시설 갇힘 임산부 철도사고 산업재해 붉은불개미 식중독 자전거사고 레저사고 인명사고 주취 담뱃불 불티 스파크 용접 라이터불 성냥불 낙뢰 누전 담배꽁초 불장난 실화 쓰레기 소각 난로 연기흡입 과열 방화 발화 배선 온풍기 부주의 합선 가연성 병충해 자살 재난 안전 전염 산악 불 유류 실족 가스레인지 발열 선박 가스 압사 감염 동파 한랭질환자 학대 전기 바이러스 고압 전류 브레이크잠김방지시스템 결함 담뱃갑 폭우 방임 제동장치 학대 야생보호 위험물 몸살 누수 기저질환 과다복용 유출 충격 독극물 액화석유가스 폐렴 감전 익사 해안사고 피탄지 강추위 전복 배터리 낙석 비 미열 다이너마이트 금속화재 빗길 악천후 근육통 음주운전 과로사 적조 녹조 고장 참사 쇼크 방사능 폭설 불씨 뇌색전 수난사고 하론 무단횡단 건조 블랙아이스 빙판길 뇌졸중 불발탄 폭풍 고립 난파 페놀 범람 추돌 경상 중상 부상 쓰나미 조난 화염 쏘임 화마 미끄러짐 넘어짐 시너 극단선택 리튬 메탄 탱크로리 여진 결빙 트리클로로메탄 강진 토치 소방드론 촛불 뇌경색 졸음운전 부정맥 탈진 산악실종 알킬레이트 수질사고 낙하물 폭죽 출혈 열사병 필로폰 연탄가스 열돔 장마 일사병 사이안화수소 투신 싱크홀 포트홀 자일렌 황화수소 맨홀 철거 아세톤 LNG",
            "operator":"OR"
        }}}
        ],
        "must_not": [
          {"match": {
         "YNewsML.NewsContent.Body": {
           "query":"전쟁",
           "operator":"OR"}}}
        ],
        "filter": [{"range": {
      "send_timestamp": {
        "gte": "2021-08-11T00:00:00.000Z",
        "lte": "2022-11-28T23:59:59.999Z",
        "format": "strict_date_optional_time"
      }}
    },{"terms": {
          "YNewsML.Metadata.Class.ClassCode.code": [
            "606001",
            "606002",
            "606003",
            "606004",
            "606005",
            "606006",
            "606007",
            "606008",
            "606009",
            "606010",
            "606011",
            "606012",
            "606013",
            "607001",
            "607002",
            "607003",
            "607004",
            "607005",
            "607006"
          ]
        }}]
  }}}
    return body

In [ ]:
size = 10000
index ='rens_test'
df = pd.DataFrame()
for i in tqdm(range(0, total+size, size)):
        result = es.search(index= index, sort='send_timestamp', body=make_body(start=i, size=size), request_timeout=30)
        result1 = Select.from_dict(result).to_pandas()
        df = pd.concat([df,result1])

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df6 = df[['YNewsML.NewsContent.Title','YNewsML.NewsContent.Body','NamedEntity']]

In [ ]:
all2 = pd.DataFrame(df6.NamedEntity.dropna()).reset_index().drop('index',axis=1)

In [ ]:
EV_word_all2 = []
for i in range(len(all2)):
    for j in range(len(all2['NamedEntity'][i])):
        if all2['NamedEntity'][i][j]['code'] == 'EV':
            EV_word_all2.append(all2['NamedEntity'][i][j]['word'])

In [ ]:
EV_word_all2_list = pd.DataFrame(pd.DataFrame(EV_word_all2).value_counts()).rename(columns={0:'counts'}).reset_index().rename(columns={0:'키워드'})
EV_word_all2_list

In [ ]:
def clean(body):
    # body = re.sub(r'[^ A-Za-z가-힣0-9+]',' ' , body, 0, re.I|re.S) # 한글, 숫자, 알파벳을 제외한 나머지 제거
    body = re.sub('태풍', '', body)
    return body

In [ ]:
EV_word_all2_list['키워드'] =  EV_word_all2_list['키워드'].progress_apply(clean)

In [ ]:
EV_word_all2_1 = EV_word_all2_list.loc[EV_word_all2_list['키워드'] != ''].sort_values(by='키워드').reset_index().drop('index',axis=1)

In [ ]:
EV_word_all2_1_group = EV_word_all2_1.groupby('키워드').sum().reset_index().sort_values('counts',ascending =False).reset_index().drop('index',axis=1)

In [ ]:
len(EV_word_all2_1_group)

In [ ]:
EV_word_all2_1_group.to_csv('count_month_withoutwar.csv',encoding='utf-8-sig',index=False)

In [ ]:
EV_word_all2_1_group.loc[EV_word_all2_1_group.counts > 10]

In [ ]:
def make_body(start, size=1000):
    #쿼리 정의 (전체 데이터 불러오기) 
    body= {
        "track_total_hits" : True,
        "from": start,
        "size": start+size, 
        "query": {
    "bool":{
        "filter": [{"range": {
      "send_timestamp": {
        "gte": "2021-08-11T00:00:00.000Z",
        "format": "strict_date_optional_time"
      }}
    },{"terms": {
          "YNewsML.Metadata.Class.ClassCode.code": [
            "606001",
            "606002",
            "606003",
            "606004",
            "606005",
            "606006",
            "606007",
            "606008",
            "606009",
            "606010",
            "606011",
            "606012",
            "606013",
            "607001",
            "607002",
            "607003",
            "607004",
            "607005",
            "607006"
          ]
        }}]
  }}}
    return body

In [ ]:
index = 'rens_test'
size = 1000
#Data 전체 수 가져오기
total = es.search(index= index, body=make_body(start=0))['hits']['total']['value']
#Data 형태 보기
total1 = es.search(index= index, body=make_body(start=0))

total

In [ ]:
size = 10000
index ='rens_test'
df = pd.DataFrame()
for i in tqdm(range(0, total+size, size)):
        result = es.search(index= index, sort='send_timestamp', body=make_body(start=i, size=size), request_timeout=30)
        result1 = Select.from_dict(result).to_pandas()
        df = pd.concat([df,result1])

In [ ]:
df1 = df[['YNewsML.NewsContent.Title','YNewsML.NewsContent.Body','NamedEntity']].reset_index().drop('index',axis=1)

In [ ]:
df1['NamedEntity'][0]

In [ ]:
all2 = pd.DataFrame(df1.NamedEntity.dropna()).reset_index().drop('index',axis=1)

In [ ]:
len(all2)

In [ ]:
EV_word_all2 = []
for i in range(len(all2)):
    for j in range(len(all2['NamedEntity'][i])):
        if all2['NamedEntity'][i][j]['code'] == 'EV':
            EV_word_all2.append(all2['NamedEntity'][i][j]['word'])

In [ ]:
df2 = pd.DataFrame(pd.DataFrame(EV_word_all2).value_counts().index.tolist())

In [ ]:
df3= df2[0].progress_apply(clean).value_counts().index.tolist()

In [ ]:
pd.DataFrame(df3).rename(columns={0:'키워드'}).to_csv('cleansing_counts.csv',encoding='utf-8-sig',index=False)

## 6개월 치

In [ ]:
def make_body(start, size=1000):
    #쿼리 정의 (전체 데이터 불러오기) 
    body= {
        "track_total_hits" : True,
        "from": start,
        "size": start+size,
        "sort": [
  {
    "send_timestamp": {
      "order": "desc"
    }
  }
], 
        "query": {
    "bool":{
      "filter": [
        {
          "exists": {
            "field": "NamedEntity"
          }
        }, 
        {
          "range": {
            "send_timestamp": {
              "gte": "2021-08-11T00:00:00.000Z",
              "format": "strict_date_optional_time"
            }
          }
        }
        
        ,{"terms": {
          "YNewsML.Metadata.Class.ClassCode.code": [
            "606001",
            "606002",
            "606003",
            "606004",
            "606005",
            "606006",
            "606007",
            "606008",
            "606009",
            "606010",
            "606011",
            "606012",
            "606013",
            "607001",
            "607002",
            "607003",
            "607004",
            "607005",
            "607006"
          ]
        }}
        
      ]
  }}}
    return body

In [ ]:
index = 'rens_test'
size = 1000
#Data 전체 수 가져오기
total = es.search(index= index, body=make_body(start=0))['hits']['total']['value']
#Data 형태 보기
total1 = es.search(index= index, body=make_body(start=0))

total

In [ ]:
size = 10000
index ='rens_test'
df = pd.DataFrame()
for i in tqdm(range(0, total+size, size)):
        result = es.search(index= index, sort='send_timestamp', body=make_body(start=i, size=size), request_timeout=30)
        result1 = Select.from_dict(result).to_pandas()
        df = pd.concat([df,result1])

In [ ]:
df.columns

In [ ]:
df1 = df[['YNewsML.NewsContent.Title','YNewsML.NewsContent.Body','NamedEntity','send_timestamp']].reset_index().drop('index',axis=1)

In [ ]:
df1

In [ ]:
def month(data):
    data = pd.to_datetime(str(data))
    return data.month

In [ ]:
def year(data):
    data = pd.to_datetime(str(data))
    return data.year

In [ ]:
def day(data):
    data = pd.to_datetime(str(data))
    return data.day

In [ ]:
from datetime import datetime, timedelta
def get_date(date):
    return pd.to_datetime(date)

In [ ]:
def get_week_no(date):
    target = get_date(date)
    firstday = target.replace(day=1)
    if firstday.weekday() == 6:
        origin = firstday
    elif firstday.weekday() < 3:
        origin = firstday - timedelta(days=firstday.weekday() + 1)
    else:
        origin = firstday + timedelta(days=6-firstday.weekday())
    return (target - origin).days // 7 + 1

In [ ]:
df1['year'] = df1.send_timestamp.apply(year)
df1['month'] = df1.send_timestamp.apply(month)
df1['day'] = df1.send_timestamp.apply(day)
df1['week'] = df1.send_timestamp.apply(get_week_no)

In [ ]:
cond1 = df1.month == 6
cond2 = df1.month == 7
cond3 = df1.month == 8
cond4 = df1.month == 9
cond5 = df1.month == 10
cond6 = df1.month == 11
cond7 = df1.month == 12

In [ ]:
d = [cond1,cond2,cond3,cond4,cond5,cond6,cond7]


In [ ]:
for l in d:
        all3 = pd.DataFrame(df1.loc[l][['NamedEntity','year','month','YNewsML.NewsContent.Title']]).reset_index().drop('index',axis=1)
        EV_word_all3 = []
        title_all3 = []
        for i in range(len(all3)):
            for j in range(len(all3['NamedEntity'][i])):
                if all3['NamedEntity'][i][j]['code'] == 'EV':
                    EV_word_all3.append(all3['NamedEntity'][i][j]['word'])
                    title_all3.append(all3['YNewsML.NewsContent.Title'][i])
        EV_word_all3 = pd.DataFrame([EV_word_all3,title_all3])
        print(len(all3))
        a = pd.DataFrame(EV_word_all3).T.rename(columns={0:'word'}).rename(columns={1:'title'})
        a['month'] = all3['month']
        globals()[f"result_{len(all3)}"] = a

In [ ]:
cond1_result = result_800.rename(columns = {'word':'키워드1','count':'count1','month':'month1','title':'title1'})
cond2_result = result_952.rename(columns = {'word':'키워드2','count':'count2','month':'month2','title':'title2'})
cond3_result = result_2545.rename(columns = {'word':'키워드3','count':'count3','month':'month3','title':'title3'})
cond4_result = result_2344.rename(columns = {'word':'키워드4','count':'count4','month':'month4','title':'title4'})
cond5_result = result_2309.rename(columns = {'word':'키워드5','count':'count5','month':'month5','title':'title5'})
cond6_result = result_2969.rename(columns = {'word':'키워드6','count':'count6','month':'month6','title':'title6'})
cond7_result = result_1555.rename(columns = {'word':'키워드7','count':'count7','month':'month7','title':'title7'})

In [ ]:
df10 = pd.concat([cond1_result,cond2_result,cond3_result,cond4_result,cond5_result,cond6_result,cond7_result],axis=1)

In [ ]:
df10

In [ ]:
df10.to_csv("with_timeline.csv",encoding='utf-8-sig', index=False)

In [ ]:
cond1_1 = df1.week == 1
cond2_1 = df1.week == 2
cond3_1 = df1.week == 3
cond4_1 = df1.week == 4
cond5_1 = df1.week == 5
cond6_1 = df1.week == 6

In [ ]:
d_1 = [cond1_1,cond2_1,cond3_1,cond4_1,cond5_1,cond6_1]

In [ ]:
for l in d:
        all3 = pd.DataFrame(df1.loc[l][['NamedEntity','year','month']]).reset_index().drop('index',axis=1)
        EV_word_all3 = []
        for i in range(len(all3)):
            for j in range(len(all3['NamedEntity'][i])):
                if all3['NamedEntity'][i][j]['code'] == 'EV':
                    EV_word_all3.append(all3['NamedEntity'][i][j]['word'])
        EV_word_all3 = pd.DataFrame(EV_word_all3)
        print(len(all3))
        a = pd.DataFrame(EV_word_all3.value_counts()).rename(columns={0:'count'}).reset_index().rename(columns={0:'키워드'})
        a['month'] = all3['month']
        globals()[f"result_{len(all3)}"] = a

In [ ]:
for l in d_1:
        try:
            all3 = pd.DataFrame(df1.loc[cond2&l][['NamedEntity','week','month']]).reset_index().drop('index',axis=1)
            EV_word_all3 = []
            for i in range(len(all3)):
                for j in range(len(all3['NamedEntity'][i])):
                    if all3['NamedEntity'][i][j]['code'] == 'EV':
                        EV_word_all3.append(all3['NamedEntity'][i][j]['word'])
            EV_word_all3 = pd.DataFrame(EV_word_all3)
            print(len(all3))
            a = pd.DataFrame(EV_word_all3.value_counts()).rename(columns={0:'count'}).reset_index().rename(columns={0:'키워드'})
            a['month'] = all3['month']
            a['week'] = all3['week']
            globals()[f"result_{len(all3)}"] = a
        except:
            pass

In [ ]:
cond2_1_result = result_216.rename(columns = {'키워드':'키워드1','count':'count1','month':'month1','week':'week1'})
cond2_2_result = result_221.rename(columns = {'키워드':'키워드2','count':'count2','month':'month2','week':'week2'})
cond2_3_result = result_192.rename(columns = {'키워드':'키워드3','count':'count3','month':'month3','week':'week3'})
cond2_4_result = result_186.rename(columns = {'키워드':'키워드4','count':'count4','month':'month4','week':'week4'})
cond2_5_result = result_21.rename(columns = {'키워드':'키워드5','count':'count5','month':'month5','week':'week5'})
# cond3_6_result = result_21.rename(columns = {'키워드':'키워드6','count':'count6','month':'month6','week':'week6'})

In [ ]:
df1_1 = pd.concat([cond1_2_result,cond1_3_result,cond1_4_result,cond1_5_result],axis=1)
df2_1 = pd.concat([cond2_1_result,cond2_2_result,cond2_3_result,cond2_4_result,cond2_5_result],axis=1)
df3_1 = pd.concat([cond3_1_result,cond3_2_result,cond3_3_result,cond3_4_result,cond3_5_result],axis=1)
df4_1 = pd.concat([cond4_1_result,cond4_2_result,cond4_3_result,cond4_4_result],axis=1)
df5_1 = pd.concat([cond5_1_result,cond5_2_result,cond5_3_result,cond5_4_result,cond5_5_result],axis=1)
df6_1 = pd.concat([cond6_1_result,cond6_2_result,cond6_3_result,cond6_4_result,cond6_5_result],axis=1)
df7_1 = pd.concat([cond7_1_result,cond7_2_result,cond7_3_result,cond7_4_result],axis=1)
# df1 = pd.concat([cond1_result,cond2_result,cond3_result,cond4_result,cond5_result,cond6_result,cond7_result],axis=1)

In [ ]:
df7_1.to_csv("with_timeline_12.csv",encoding='utf-8-sig', index=False)

In [ ]:
def make_body(start, size=1000):
    #쿼리 정의 (전체 데이터 불러오기) 
    body= {
        "track_total_hits" : True,
        "from": start,
        "size": start+size, 
        "query": {
    "bool":{ 
        "must_not": [
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "0801002"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "0402004"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "607001"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "607002"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "607003"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "607004"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "607005"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "607006"
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "태풍"
        }}],
      "filter": [
        {
          "exists": {
            "field": "NamedEntity"
          }
        }, 
        {
          "range": {
            "send_timestamp": {
              "gte": "2021-08-11T00:00:00.000Z",
              "format": "strict_date_optional_time"
            }
          }
        },
        {"terms": {
          "YNewsML.Metadata.Class.ClassCode.code": [
            "606001",
            "606002",
            "606003",
            "606004",
            "606005",
            "606006",
            "606007",
            "606009",
            "606010",
            "606011",
            "606012",
            "606013"
          ]
        }}
        
      ]
  }}}
    return body

In [ ]:
index = 'rens_test'
size = 1000
#Data 전체 수 가져오기
total = es.search(index= index, body=make_body(start=0))['hits']['total']['value']
#Data 형태 보기
total1 = es.search(index= index, body=make_body(start=0))

total

In [ ]:
size = 10000
index ='rens_test'
df = pd.DataFrame()
for i in tqdm(range(0, total+size, size)):
        result = es.search(index= index, sort='send_timestamp', body=make_body(start=i, size=size), request_timeout=30)
        result1 = Select.from_dict(result).to_pandas()
        df = pd.concat([df,result1])

In [ ]:
df2 = df

In [ ]:
df2 = df2.reset_index().drop('index',axis=1)

In [ ]:
EV_word_all2 = []
for i in range(len(df2)):
    for j in range(len(df2['NamedEntity'][i])):
        if df2['NamedEntity'][i][j]['code'] == 'EV':
            EV_word_all2.append([df2['NamedEntity'][i][j]['word'],df2['YNewsML.NewsContent.Title'][i]])

In [ ]:
pd.DataFrame(EV_word_all2)

In [ ]:
df2 = pd.DataFrame(EV_word_all2).rename(columns={0:'words',1:'title'}).drop_duplicates().reset_index().drop('index',axis=1)
df2

In [ ]:
# df2 = pd.DataFrame(pd.DataFrame(EV_word_all2).value_counts()).rename(columns={0:'counts'}).reset_index().rename(columns={0:'words'})
# len(df2)

In [ ]:
df2.to_csv('only_acc_code1.csv',encoding='utf-8-sig',index=False)

In [ ]:
df.columns

In [ ]:
df1 = df[['YNewsML.NewsContent.Title','YNewsML.NewsContent.Body','Keyword','send_timestamp','YNewsML.Metadata.Class.ClassCode']].reset_index().drop('index',axis=1)

In [ ]:
a =[]
for i in df1['Keyword']:
    for j in range(len(i)):
        a.append(i[j])

In [ ]:
b = pd.DataFrame(pd.DataFrame(a).value_counts()).rename(columns={0:'counts'}).reset_index().rename(columns={0:'사고유형'})
b

In [ ]:
b.describe()

In [ ]:
b = b[b['counts'] > b['counts'].mean()]
b

In [ ]:
def clean(body):
    body = re.sub(r'[^가-힣+]',' ' , body, 0, re.I|re.S) # 한글을 제외한 나머지 제거
    body = re.sub(' ','' , body, 0, re.I|re.S) # 공백 제거
    return body

In [ ]:
b['사고유형'] = b['사고유형'].apply(clean)

In [ ]:
c=[]
for i in b['사고유형']:
    if len(i) > 1:
        c.append(i)

In [ ]:
d = pd.DataFrame(pd.DataFrame(c).value_counts()).rename(columns={0:'counts'}).reset_index().rename(columns={0:'word'})['word']

In [ ]:
d = d.tolist()

In [ ]:
from konlpy.tag import Okt

In [ ]:
okt = Okt()

In [ ]:
d

In [ ]:
noun_list=[]
for i in d:
    noun = okt.nouns(i)
    noun_list.append(noun)

In [ ]:
p = []
for i in noun_list:
    t=''
    for j in range(len(i)):
            t += i[j]
    p.append(t)

In [ ]:
p = pd.DataFrame(p)
p = p[p != ''].dropna()

In [ ]:
p = p[0].tolist()

In [ ]:
p

In [ ]:
inter = [list(set(i).intersection(p)) for i in df2['Keyword']]

In [ ]:
inter_num = []
for i in inter:
    inter_num.append(len(i))

In [ ]:
inter_result = pd.DataFrame([inter, inter_num]).T.rename(columns={0:'words',1:'count'})

In [ ]:
inter_result

In [ ]:
inter_index = inter_result[inter_result['count'] >= 2].index.tolist()

In [ ]:
len(inter_index)

In [ ]:
inter_final = []
for i in inter_index:
    inter_final.append([df2.loc[i]])

In [ ]:
result = pd.DataFrame()
for i in inter_final:
    result = pd.concat([result,pd.DataFrame(i)])

In [ ]:
result['YNewsML.Metadata.Class.ClassCode'].tolist()

In [ ]:
result.to_csv('화재뉴스.csv',encoding='utf-8-sig',index=False)

In [ ]:
result = result.reset_index().drop('index',axis=1)

In [ ]:
EV_word_all2 = []
for i in range(len(result)):
    for j in range(len(result['NamedEntity'][i])):
        if result['NamedEntity'][i][j]['code'] == 'EV':
            EV_word_all2.append(result['NamedEntity'][i][j]['word'])

In [ ]:
pd.DataFrame(pd.DataFrame(EV_word_all2).value_counts()).rename(columns={0:'counts'}).reset_index().rename(columns={0:'word'}).to_csv('화재키워드.csv',encoding='utf-8-sig',index=False)

In [ ]:
big = pd.read_csv('/home/intern2/fire_dev/big.csv', encoding='utf-8-sig')

In [ ]:
big

In [ ]:
EV_word_big = []
for i in range(len(big)):
    for j in range(len(big['NER'][i])):
        if big['NER'][i][j]['code'] == 'EV':
            EV_word_big.append(big['NER'][i][j]['word'])

## 자연재해

In [ ]:
def make_body(start, size=1000):
    #쿼리 정의 (전체 데이터 불러오기) 
    body= {
        "track_total_hits" : True,
        "from": start,
        "size": start+size, 
        "query": {
    "bool":{ 
        "must_not": [
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606001"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606002"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606003"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606004"
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606005"            
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606006" 
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606007" 
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606009" 
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606010" 
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606011" 
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606012" 
        }},
        {"match": {
          "YNewsML.Metadata.Class.ClassCode.code": "606013" 
        }},
        {"match": {
          "YNewsML.NewsContent.Title": "훈련" 
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "축제" 
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "우크라이나" 
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "대회" 
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "대전" 
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "회담" 
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "월드컵" 
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "봉사" 
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "협약" 
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "기후위기" 
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "엑스포" 
        }},
        {"match": {
          "YNewsML.NewsContent.Body": "경기" 
        }}],
      "filter": [
        {
          "exists": {
            "field": "NamedEntity"
          }
        }, 
        {
          "range": {
            "send_timestamp": {
              "gte": "2021-08-11T00:00:00.000Z",
              "format": "strict_date_optional_time"
            }
          }
        },
        {"terms": {
          "YNewsML.Metadata.Class.ClassCode.code": [
            "607001",
            "607002",
            "607003",
            "607004",
            "607005",
            "607006"
          ]
        }}
        
      ]
  }}}
    return body

In [ ]:
index = 'rens_test'
size = 1000
#Data 전체 수 가져오기
total = es.search(index= index, body=make_body(start=0))['hits']['total']['value']
#Data 형태 보기
total1 = es.search(index= index, body=make_body(start=0))

total

In [ ]:
size = 10000
index ='rens_test'
df = pd.DataFrame()
for i in tqdm(range(0, total+size, size)):
        result = es.search(index= index, sort='send_timestamp', body=make_body(start=i, size=size), request_timeout=30)
        result1 = Select.from_dict(result).to_pandas()
        df = pd.concat([df,result1])

In [ ]:
df2 = df

In [ ]:
EV_word_all2 = []
for i in range(len(df2)):
    for j in range(len(df2['NamedEntity'][i])):
        if df2['NamedEntity'][i][j]['code'] == 'EV':
            EV_word_all2.append([df2['NamedEntity'][i][j]['word'],df2['YNewsML.NewsContent.Title'][i]])

In [ ]:
df3 = pd.DataFrame(EV_word_all2).drop_duplicates().reset_index().drop('index',axis=1).rename(columns={0:'word',1:'title'})

In [ ]:
df3

In [ ]:
df3.to_csv('only_acc_code_nature.csv',encoding='utf-8-sig',index=False)